In [1]:
import os
from pathlib import Path
import sys
from tqdm import tqdm
import numpy as np
import glob
from nnabla.utils.data_source import DataSource
from nnabla.utils.data_iterator import data_iterator


class DatasetGenerator:
    r""" Splits the Dataset and Labels into batches containing n_utterances for each of the n_speakers

    """

    def __init__(self):

        self.dev_dir = "./data/LibriSpeech/dev-clean"
        self.test_dir = "./data/LibriSpeech/test-clean"
        self.label_dir = "./data/LibriSpeech/labels/"
        self.mfcc_dev_dir = "./data/LibriSpeech/mfcc-dev/"
        self.mfcc_test_dir = "./data/LibriSpeech/mfcc-test/"
        self.n_utterances = 5  # utterances per speaker in batch
        self.batch_size = 200
        self.n_timesteps = 101
        self.n_features = 40
        self.ys = np.load(self.label_dir + 's_id.npy')
        self.xs = []
        mfccs = sorted(glob.glob(self.mfcc_dev_dir + '*.npy'))
        for i in tqdm(range(len(mfccs))):
            self.xs.append((np.load(mfccs[i])).T)
        
        self.n_data = len(self.xs)

    # Batch_size =200 , each batch will contain 5 utterances from each of the 40 speakers

    def get_features(self):
        ys = np.load(self.label_dir + 's_id.npy')

        ''' 
        Get total utterances for each speaker
        '''
        inputs = {k: [] for k in sorted(set(ys))}

        for i, s_id in enumerate(ys):
            inputs[s_id].append(self.xs[i])

        return inputs

    def prep_dataset(self):    
        mfccs = self.get_features()
        n_iters = self.n_data // self.batch_size
        total_data = n_iters*self.batch_size
        dataset = np.zeros((total_data,self.n_timesteps,self.n_features))
        for sid in mfccs:
            np.random.shuffle(mfccs[sid])            
        for i in range(0, total_data, self.batch_size):            
            for sid in mfccs:
                for j in range(self.n_utterances):                    
                    dataset[i+j] = mfccs[sid][j]
                i = i+j+1            
                mfccs[sid] = np.delete(mfccs[sid], range(self.n_utterances), axis=0)
                mfccs[sid] = np.append(mfccs[sid], dataset[-self.n_utterances:], axis=0)
        return dataset

2021-01-24 22:08:17,108 [nnabla][INFO]: Initializing CPU extension...


In [2]:
dg = DatasetGenerator()
dataset = dg.prep_dataset()

100%|██████████████████████████████████████████████████████████████████████████████| 7310/7310 [01:15<00:00, 97.19it/s]


In [3]:
import os
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import nnabla as nn
import nnabla.parametric_functions as PF
import nnabla.functions as F
import nnabla.solvers as S
from nnabla.utils.data_iterator import data_iterator_simple
import nnabla.monitor as M
import nnabla.initializer as I

In [6]:
class Encoder:

    def __init__(self,dataset):
        '''
        Model Params (to be moved to hparams.py)
        '''
        self.n_utterances = 5
        self.n_speakers = 40
        self.batch_size = 200
        self.lstm_layers = 3
        self.lstm_hidden = 256
        self.lstm_directions = 1
        self.affine_hidden = 256
        self.embed_size = 256
        self.eps = 1e-6
        self.dataset = dataset 
#         DatasetGenerator().split_dataset()

        self.sim_weight = 0.0 
        self.sim_bias = 0.0
        self.sim_matrix = 0.0 
        self.eps = 1e-5

    def get_batch(self,i):
        '''
        Divide the dataset into batches containing 'm' utterances for 'n' speakers
        '''

        batch_size = self.batch_size
        batch_data = self.dataset[i*batch_size:(i+1)*batch_size]
#         self.dataset = np.delete(self.dataset, range(batch_size))
#         self.dataset = np.concatenate((self.dataset,batch_data), axis = 0)
        # convert shape from (B,T,I)  -> (T,B,I)  ((200, 138, 13) -> (138, 200, 13))
        batch_data = np.transpose(batch_data, (1, 0, 2))

        return batch_data

    def encoder(self, inputs, training=True):
        '''
        Speaker Encoder network
        '''

        L = self.lstm_layers
        D = self.lstm_directions
        B = self.batch_size
        H = self.lstm_hidden
        T, _, I = inputs.shape
        h = nn.Variable(shape=(L, D, B, H), need_grad=True)
        c = nn.Variable(shape=(L, D, B, H), need_grad=True)
        h.d = np.zeros(h.shape)
        c.d = np.zeros(c.shape)

        y, hn, cn = PF.lstm(inputs, h, c, num_layers=L, training=training)
        
        out = PF.affine(hn[-1][0], self.affine_hidden)
        out = F.relu(out)
        out = out/(F.norm(out,axis=1).reshape((-1,1)) + self.eps)
        
#        print(out.d)

        return out

    def similarity_matrix(self,emb):
        N = self.n_speakers
        M = self.n_utterances
        P = self.embed_size
        
        emb_re = emb.reshape((N, M, P))

        # compute the inclusion centroids
        cen = F.mean(emb_re, axis=1) 
        cen = cen / F.norm(cen, axis=1, keepdims=True)
        cen = F.reshape(cen, (-1, P))

        # compute the exclusion centroids
        exc = F.sum(emb_re, axis=1, keepdims=True) - emb_re
        exc = exc / (M - 1)
        exc = exc / F.norm(exc, axis=2, keepdims=True)
        exc = F.reshape(exc, emb.shape)

        diag = F.sum(exc * emb, axis=1, keepdims=True) # 20 x 1
        sim = F.affine(emb,  F.transpose(cen, (1, 0))) 

        mask = np.concatenate([np.tile(w, (M, 1)) for w in np.eye(N)])
        mask = nn.Variable.from_numpy_array(mask)
        sm = (1 - mask) * sim + F.tile(diag, N) * mask
        #sm = self.sim_weight*sm + self.sim_bias

        sm = F.add_scalar(F.mul_scalar(sm, self.sim_weight.d), self.sim_bias.d)
        
        return sm

#     def similarity_matrix(self, embeddings):
#         """
#             Computes the similarity matrix according the section 2.1 of GE2E.
#         """
#         N = self.n_speakers
#         M = self.n_utterances
#         P = self.embed_size
#         embeddings = embeddings.reshape(
#             (self.n_speakers, self.n_utterances, self.embed_size))
       
#         center = F.mean(embeddings, axis=1)             # [N,P] normalized center vectors eq.(1)
#         center_norm = F.norm(center,2,keepdims=True)
#         center = center/center_norm
#         center_except = F.reshape(F.sum(embeddings, axis=1, keepdims=True) - embeddings, (N*M,P))  # [NM,P] center vectors eq.(8)
#         # make similarity matrix eq.(9)
                
#         sm = np.concatenate(
#             [np.concatenate([np.sum(center_except.d[i*M:(i+1)*M,:]*embeddings.d[j,:,:], axis=1, keepdims=True) if i==j
#                         else np.sum(center.d[i:(i+1),:]*embeddings.d[j,:,:], axis=1, keepdims=True) for i in range(N)],
#                        axis=1) for j in range(N)], axis=0)
#         self.sim_matrix = nn.Variable.from_numpy_array(sm)
#         self.sim_matrix = F.add_scalar(F.mul_scalar(self.sim_matrix, self.sim_weight.d), self.sim_bias.d)
#         #print(self.sim_matrix.d)
#         return nn.Variable.from_numpy_array(sm)
        
    def loss_fn(self, embeddings):
        '''
        Define a cross entropy loss between the similarity matrix and the ground truth labels
        '''
        sim_matrix = self.similarity_matrix(embeddings)
        #self.sim_matrix.data.copy_from(sm.data)
        #self.sim_matrix = self.sim_matrix.reshape(
        #    (self.n_speakers * self.n_utterances, self.n_speakers))

        # Create ground truth labels
        ground_truth = nn.Variable.from_numpy_array(
            np.repeat(np.arange(self.n_speakers), self.n_utterances))
        ground_truth = ground_truth.reshape(
            (self.n_speakers * self.n_utterances, 1))

        loss = F.softmax_cross_entropy(sim_matrix, ground_truth)
        
        return loss
          

    def train(self):
        nn.clear_parameters()
        #nn.set_auto_forward(True)
        mfcc_dim1, mfcc_dim2 = self.dataset[0].shape
        n_batch = len(self.dataset) // self.batch_size
        max_epochs = 600
        loss_scale = 8
        monitor = M.Monitor('.')
        monitor_loss = M.MonitorSeries(
            "Training loss", monitor, interval=2, verbose=True)
        monitor_time = M.MonitorTimeElapsed(
            "Training time", monitor, interval=1000, verbose=True)
        self.sim_weight = nn.parameter.get_parameter_or_create(
            'sim_weight', (1,), I.ConstantInitializer(10.0), need_grad=True)
        self.sim_bias = nn.parameter.get_parameter_or_create(
             'sim_bias', (1,), I.ConstantInitializer(-5.0), need_grad=True)
        
#         self.sim_matrix = nn.Variable.from_numpy_array(np.zeros((self.n_speakers * self.n_utterances, self.n_speakers)), 
#                                                        need_grad = True)

        solver = S.Adam()
        xi = nn.Variable((mfcc_dim1, self.batch_size, mfcc_dim2), need_grad=True)
        # Get the embeddings from the encoder
        embeddings = self.encoder(xi, training=True)
        loss = self.loss_fn(embeddings) # Define loss
        solver.set_parameters(nn.get_parameters())
        print(nn.get_parameters())

        for epoch in range(max_epochs):
            
            # Iterations per epoch

            for i in range(n_batch):

                # Returns current batch
                xi.d = self.get_batch(i)

                loss.forward()
                solver.zero_grad()
                loss.backward(clear_buffer=True)
                solver.scale_grad(1. / loss_scale)
                solver.update()

                # monitor
                itr = epoch * n_batch + i
                monitor_loss.add(itr, loss.d)
            print(f"Epoch:{epoch} | Loss:{loss.d}")

In [7]:
enc = Encoder(dataset)
enc.train()

OrderedDict([('sim_weight', <Variable((1,), need_grad=True) at 0x2996100b450>), ('sim_bias', <Variable((1,), need_grad=True) at 0x29961043810>), ('lstm/weight_l0', <Variable((1, 4, 256, 296), need_grad=True) at 0x299612fa6d0>), ('lstm/weight', <Variable((2, 1, 4, 256, 512), need_grad=True) at 0x299612faa90>), ('lstm/bias', <Variable((3, 1, 4, 256), need_grad=True) at 0x299612fabd0>), ('affine/W', <Variable((256, 256), need_grad=True) at 0x299612faae0>), ('affine/b', <Variable((256,), need_grad=True) at 0x299612fac70>)])


2021-01-24 22:11:19,776 [nnabla][INFO]: iter=1 {Training loss}=3.5063488483428955
2021-01-24 22:11:38,153 [nnabla][INFO]: iter=3 {Training loss}=3.296698570251465
2021-01-24 22:11:57,328 [nnabla][INFO]: iter=5 {Training loss}=3.2159783840179443
2021-01-24 22:12:16,760 [nnabla][INFO]: iter=7 {Training loss}=3.0217413902282715
2021-01-24 22:12:36,028 [nnabla][INFO]: iter=9 {Training loss}=2.9759716987609863
2021-01-24 22:12:55,646 [nnabla][INFO]: iter=11 {Training loss}=2.984682559967041
2021-01-24 22:13:14,974 [nnabla][INFO]: iter=13 {Training loss}=3.074453353881836
2021-01-24 22:13:33,942 [nnabla][INFO]: iter=15 {Training loss}=2.9013309478759766
2021-01-24 22:13:52,806 [nnabla][INFO]: iter=17 {Training loss}=2.781064987182617
2021-01-24 22:14:11,942 [nnabla][INFO]: iter=19 {Training loss}=2.9247536659240723
2021-01-24 22:14:30,685 [nnabla][INFO]: iter=21 {Training loss}=2.72047758102417
2021-01-24 22:14:49,615 [nnabla][INFO]: iter=23 {Training loss}=2.7683305740356445
2021-01-24 22:1

Epoch:0 | Loss:[[ 1.1343552 ]
 [ 0.93748546]
 [ 1.2255394 ]
 [ 0.9675657 ]
 [ 0.9978937 ]
 [ 2.2459593 ]
 [ 1.6836097 ]
 [ 1.6575428 ]
 [ 1.668905  ]
 [ 1.6626154 ]
 [ 3.1590657 ]
 [ 2.4845529 ]
 [ 2.3917468 ]
 [ 4.165096  ]
 [ 4.165096  ]
 [ 1.4441309 ]
 [ 3.8939636 ]
 [ 1.4441309 ]
 [ 2.8362255 ]
 [ 1.7490937 ]
 [ 1.5424505 ]
 [ 1.4881225 ]
 [ 1.3491092 ]
 [ 1.3916821 ]
 [ 1.5216861 ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 6.0893693 ]
 [ 5.1690626 ]
 [ 5.9380436 ]
 [ 2.039061  ]
 [ 2.19873   ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.838387  ]
 [ 2.1456673 ]
 [ 1.938043  ]
 [ 1.7798697 ]
 [ 2.138361  ]
 [ 4.027737  ]
 [ 2.83838

2021-01-24 22:17:01,334 [nnabla][INFO]: iter=37 {Training loss}=2.7550432682037354
2021-01-24 22:17:20,302 [nnabla][INFO]: iter=39 {Training loss}=2.6989781856536865
2021-01-24 22:17:39,074 [nnabla][INFO]: iter=41 {Training loss}=2.555396795272827
2021-01-24 22:17:57,980 [nnabla][INFO]: iter=43 {Training loss}=2.630783796310425
2021-01-24 22:18:17,094 [nnabla][INFO]: iter=45 {Training loss}=2.7999613285064697
2021-01-24 22:18:36,039 [nnabla][INFO]: iter=47 {Training loss}=2.5470457077026367
2021-01-24 22:18:54,786 [nnabla][INFO]: iter=49 {Training loss}=2.7106215953826904
2021-01-24 22:19:13,417 [nnabla][INFO]: iter=51 {Training loss}=2.56636118888855
2021-01-24 22:19:32,242 [nnabla][INFO]: iter=53 {Training loss}=2.3179919719696045
2021-01-24 22:19:51,269 [nnabla][INFO]: iter=55 {Training loss}=2.56302547454834
2021-01-24 22:20:10,227 [nnabla][INFO]: iter=57 {Training loss}=2.58589243888855
2021-01-24 22:20:28,836 [nnabla][INFO]: iter=59 {Training loss}=2.513296604156494
2021-01-24 22

Epoch:1 | Loss:[[1.3675661 ]
 [1.2616485 ]
 [1.3119102 ]
 [1.2453952 ]
 [1.2799665 ]
 [4.1200924 ]
 [1.4924916 ]
 [1.6246169 ]
 [1.9535486 ]
 [1.9175243 ]
 [3.0902555 ]
 [3.1477802 ]
 [3.679193  ]
 [6.6379538 ]
 [6.6379538 ]
 [1.9281788 ]
 [1.879652  ]
 [1.9281788 ]
 [1.7137048 ]
 [2.222343  ]
 [2.2563443 ]
 [0.55323267]
 [0.90098035]
 [0.5774286 ]
 [0.87513953]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.370552  ]
 [2.2622638 ]
 [3.4437957 ]
 [3.1152973 ]
 [1.6743511 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.1080291 ]
 [2.3940406 ]
 [1.9225305 ]
 [2.8776238 ]
 [2.1240656 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [2.7761855 ]
 [1.9

...

Epoch:13 | Loss:[[ 0.3053896 ]
 [ 0.2865725 ]
 [ 0.3352281 ]
 [ 0.6801262 ]
 [ 0.39217782]
 [ 2.2353296 ]
 [ 0.9444619 ]
 [ 0.68917483]
 [ 0.85367936]
 [ 0.8399429 ]
 [ 2.8281317 ]
 [ 2.222795  ]
 [ 2.6369863 ]
 [ 6.6993074 ]
 [ 6.6993074 ]
 [ 1.2403482 ]
 [ 1.1170275 ]
 [ 1.2403482 ]
 [ 1.5931108 ]
 [ 1.1148431 ]
 [ 1.0489922 ]
 [ 1.1567185 ]
 [ 0.8509086 ]
 [ 1.129875  ]
 [ 1.1058496 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 0.5054905 ]
 [ 0.38230458]
 [ 1.4645227 ]
 [ 2.2695434 ]
 [ 0.50324494]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 2.7609942 ]
 [ 0.72924453]
 [ 1.6606367 ]
 [ 1.1764138 ]
 [ 1.3625606 ]
 [ 1.2766297 ]
 [ 2.7609

2021-01-24 23:29:17,139 [nnabla][INFO]: iter=505 {Training loss}=1.5910378694534302
2021-01-24 23:29:35,618 [nnabla][INFO]: iter=507 {Training loss}=1.5047122240066528
2021-01-24 23:29:53,921 [nnabla][INFO]: iter=509 {Training loss}=1.5091701745986938
2021-01-24 23:30:12,682 [nnabla][INFO]: iter=511 {Training loss}=1.371121883392334
2021-01-24 23:30:31,419 [nnabla][INFO]: iter=513 {Training loss}=1.7639758586883545
2021-01-24 23:30:50,316 [nnabla][INFO]: iter=515 {Training loss}=1.5457061529159546
2021-01-24 23:31:08,450 [nnabla][INFO]: iter=517 {Training loss}=1.5162975788116455
2021-01-24 23:31:26,896 [nnabla][INFO]: iter=519 {Training loss}=1.625199556350708
2021-01-24 23:31:45,116 [nnabla][INFO]: iter=521 {Training loss}=1.424853801727295
2021-01-24 23:32:03,448 [nnabla][INFO]: iter=523 {Training loss}=1.5448483228683472
2021-01-24 23:32:22,202 [nnabla][INFO]: iter=525 {Training loss}=1.6317050457000732
2021-01-24 23:32:40,714 [nnabla][INFO]: iter=527 {Training loss}=1.557545423507

Epoch:14 | Loss:[[0.5170929 ]
 [0.50596356]
 [0.555598  ]
 [0.95067436]
 [0.604603  ]
 [2.1101642 ]
 [1.2945197 ]
 [1.2057016 ]
 [1.1543119 ]
 [2.060532  ]
 [2.8366601 ]
 [1.8213336 ]
 [1.7814302 ]
 [6.4951897 ]
 [6.4951897 ]
 [1.5829678 ]
 [1.4180293 ]
 [1.5829678 ]
 [1.7163961 ]
 [1.5448283 ]
 [1.05458   ]
 [1.2540704 ]
 [0.87264705]
 [1.1137701 ]
 [0.9430396 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [0.27886555]
 [0.23100016]
 [0.5787486 ]
 [1.7952925 ]
 [0.73052746]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [0.8705229 ]
 [1.3952467 ]
 [1.114162  ]
 [2.9941938 ]
 [1.1525978 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [2.7656524 ]
 [1.

2021-01-24 23:34:49,699 [nnabla][INFO]: iter=541 {Training loss}=1.4901132583618164
2021-01-24 23:35:08,042 [nnabla][INFO]: iter=543 {Training loss}=1.4873501062393188
2021-01-24 23:35:26,338 [nnabla][INFO]: iter=545 {Training loss}=1.4416736364364624
2021-01-24 23:35:44,443 [nnabla][INFO]: iter=547 {Training loss}=1.394999384880066
2021-01-24 23:36:02,587 [nnabla][INFO]: iter=549 {Training loss}=1.5797622203826904
2021-01-24 23:36:20,374 [nnabla][INFO]: iter=551 {Training loss}=1.4178881645202637


KeyboardInterrupt: 

In [ ]:
speakers_per_batch, utterances_per_speaker = embeds.shape[:2]
centroids_incl = F.mean(embeds, axis=1, keepdims=True)
centroids_incl = centroids_incl / F.norm(centroids_incl, axis=2, keepdims=True)
centroids_excl = F.sum(embeds, axis=1, keepdims=True) - embeds
centroids_excl /= (utterances_per_speaker - 1)
centroids_excl = centroids_excl / F.norm(centroids_excl, axis=2, keepdims=True)
sim_matrix = nn.Variable.from_numpy_array(np.zeros((speakers_per_batch, utterances_per_speaker, speakers_per_batch),dtype=np.int))
mask_matrix = 1 - np.eye(speakers_per_batch, dtype=np.int)
for j in range(speakers_per_batch):
    mask = np.where(mask_matrix[j])[0]
    sim_matrix[mask, :, j] = F.sum(embeds[mask] * centroids_incl[j],axis=2)
    sim_matrix[j, :, j] = F.sum(embeds[j] * centroids_excl[j],axis=1)

In [ ]:
center = F.mean(embedded_split, axis=1)             # [N,P] normalized center vectors eq.(1)
center_norm = F.norm(center,2,keepdims=True)
center = center/center_norm

center_except = F.reshape(F.sum(embedded_split, axis=1, keepdims=True) - embedded_split, (N*M,P))  # [NM,P] center vectors eq.(8)
# make similarity matrix eq.(9)
S = tf.concat(
    [tf.concat([tf.reduce_sum(center_except[i*M:(i+1)*M,:]*embedded_split[j,:,:], axis=1, keep_dims=True) if i==j
                else tf.reduce_sum(center[i:(i+1),:]*embedded_split[j,:,:], axis=1, keep_dims=True) for i in range(N)],
               axis=1) for j in range(N)], axis=0)

nnabla.functions.concatenate

In [ ]:
zz = {"s1":np.array([np.random.randint(1,10,(2,4)),np.random.randint(1,10,(2,4)),
                     np.random.randint(1,10,(2,4)),np.random.randint(1,10,(2,4))]),
      "s2":np.array([np.random.randint(1,10,(2,4)),np.random.randint(1,10,(2,4)),
                     np.random.randint(1,10,(2,4)),np.random.randint(1,10,(2,4))])    
     
     }
zz

In [ ]:
xx = zz.copy()
curr= xx["s1"][0:2]
print(xx["s1"].shape)
xx["s1"]=np.delete(xx["s1"], [0,1], axis=0)
xx["s1"]=np.append(xx["s1"], curr, axis=0)
print(xx["s1"].shape)
print(curr)
xx

In [14]:
nn.set_auto_forward(True)

def compute_similarity(emb, n_speakers, n_utterances):
    # embedding reshape
    emb_re = emb.reshape((n_speakers, n_utterances, -1))
    
    # compute the inclusion centroids
    cen = F.mean(emb_re, axis=1) 
    cen = cen / F.norm(cen, axis=1, keepdims=True)
    cen = F.reshape(cen, (-1, emb.shape[-1]))
    
    # compute the exclusion centroids
    exc = F.sum(emb_re, axis=1, keepdims=True) - emb_re
    exc = exc / (n_utterances - 1)
    exc = exc / F.norm(exc, axis=2, keepdims=True)
    exc = F.reshape(exc, emb.shape)

    diag = F.sum(exc * emb, axis=1, keepdims=True) # 20 x 1
    sim = F.affine(emb,  F.transpose(cen, (1, 0))) 

    mask = np.concatenate([np.tile(w, (n_utterances, 1)) for w in np.eye(n_speakers)])
    mask = nn.Variable.from_numpy_array(mask)
    ret = (1 - mask) * sim + F.tile(diag, n_speakers) * mask
    return  ret



bz = 5
dim = 6
n_speakers = 5
n_utterances = 4

rng = np.random.RandomState(1234)
data = np.arange(bz * n_utterances * dim).reshape((bz * n_utterances, dim))

rng.randn(bz, dim)

emb = nn.Variable.from_numpy_array(data)
emb = emb / F.norm(emb, axis=1, keepdims=True)

ret =  compute_similarity(emb, n_speakers, n_utterances)

In [15]:
ret.shape

(20, 5)

In [16]:
ret.d

array([[0.8931736 , 0.85282373, 0.8417678 , 0.8371577 , 0.8346132 ],
       [0.99777466, 0.9890098 , 0.98571754, 0.98425335, 0.98342335],
       [0.9846763 , 0.99773246, 0.99611515, 0.99533224, 0.99487424],
       [0.975716  , 0.99944794, 0.9985397 , 0.99804544, 0.997745  ],
       [0.9831892 , 0.9998139 , 0.99937737, 0.9990421 , 0.99882853],
       [0.98095524, 0.9999939 , 0.99972546, 0.9994907 , 0.99933195],
       [0.9793355 , 0.99997264, 0.99988353, 0.99971807, 0.99959725],
       [0.9781107 , 0.9998819 , 0.99995685, 0.999842  , 0.99974895],
       [0.9771528 , 0.99987066, 0.99998003, 0.9999125 , 0.99984056],
       [0.97638404, 0.9998066 , 0.9999988 , 0.99995357, 0.9998982 ],
       [0.97575355, 0.99974537, 0.99999726, 0.99997735, 0.9999353 ],
       [0.9752273 , 0.99968845, 0.99998474, 0.9999905 , 0.99995965],
       [0.97478133, 0.9996364 , 0.9999811 , 0.9999952 , 0.9999757 ],
       [0.9743989 , 0.99958897, 0.9999692 , 0.99999964, 0.99998623],
       [0.9740671 , 0.99954563, 0.

In [11]:
mask = np.concatenate([np.tile(w, (4, 1)) for w in np.eye(5)])

In [13]:
1-mask

array([[0., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.]])